In [1]:
!pip install scikit-surprise==1.0.6

     |████████████████████████████████| 3.3MB 7.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.0.6-cp37-cp37m-linux_x86_64.whl size=1624104 sha256=7847cc70c38f0e2dbf16e51741879ed7eeabf7fbc5b7db4dd1441fa66c45e327
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import SVD, accuracy, Trainset
from tqdm.auto import tqdm
import json
import ast

# Load trainset

In [3]:
with open('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/trainset.json', 'r') as f:
    traindata = json.load(f)

In [4]:
traindata.keys()

dict_keys(['ur_dict', 'ir_dict', 'n_users', 'n_items', 'n_ratings', 'offset', 'rating_scale', 'raw2inner_id_users', 'raw2inner_id_items'])

In [5]:
rating_scale = tuple(traindata['rating_scale'])

In [6]:
n_users, n_items, n_ratings, offset = traindata['n_users'], traindata['n_items'], traindata['n_ratings'], traindata['offset']

In [7]:
ur_dict = {}
for key in tqdm(traindata['ur_dict'].keys()):
    ur_dict[int(key)] = [tuple(x) for x in traindata['ur_dict'][key]]
ur = defaultdict(list, ur_dict)

ir_dict = {}
for key in tqdm(traindata['ir_dict'].keys()):
    ir_dict[int(key)] = [tuple(x) for x in traindata['ir_dict'][key]]
ir = defaultdict(list, ir_dict)

In [8]:
raw2inner_id_users = {}
for key in tqdm(traindata['raw2inner_id_users'].keys()):
    raw2inner_id_users[int(key)] = traindata['raw2inner_id_users'][key]

raw2inner_id_items = {}
for key in tqdm(traindata['raw2inner_id_items'].keys()):
    raw2inner_id_items[int(key)] = traindata['raw2inner_id_items'][key]

In [9]:
trainset = Trainset(ur=ur, 
                    ir=ir, 
                    n_users=n_users, 
                    n_items=n_items, 
                    n_ratings=n_ratings, 
                    rating_scale=rating_scale, 
                    offset=offset, 
                    raw2inner_id_users=raw2inner_id_users, 
                    raw2inner_id_items=raw2inner_id_items)

# Load testset

In [10]:
with open('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/testset.json', 'r') as f:
    testset = json.load(f)
testset = [tuple(x) for x in testset['testset']]

# Train

In [11]:
model = SVD(n_factors=300, n_epochs=50, verbose=True)
model.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


# Evaluate

In [12]:
predictions = model.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 1.9159
MAE:  1.2641


1.264128457876339

In [13]:
# ndcg
testdata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Test/test_data.csv')
sum = 0.0
for _, row in tqdm(testdata.iterrows()):
    rele = np.array(ast.literal_eval(row['ratings']))
    if rele.shape[0] == 0:
        sum += 1.0
        continue
    
    if rele.sum() == 0:
        sum += 1.0
        continue
    
    pred = np.array([model.predict(row['user_id'], iid).est for iid in ast.literal_eval(row['anime_ids'])])
    dlog2 = np.log2(np.arange(rele.shape[0]) + 2)
    tmp_df = pd.DataFrame({'rele': rele, 'pred': pred})
    tmp_df = tmp_df.sort_values(by=['pred'], ascending=False)
    dcg = (tmp_df['rele'].values / dlog2).sum()

    rele[::-1].sort()
    idcg = (rele / dlog2).sum()
    sum += dcg / idcg

sum / n_users

0.9544528137696252